In [ ]:
import sys

import gurobipy as gp
import numpy as np
import pandas as pd
from joblib import load
from matplotlib import pyplot as plt

In [ ]:
# import my functions
%load_ext autoreload
%autoreload 2
sys.path.append('../../src')
from ml2gurobi.activations import Identity
from ml2gurobi.extra.morerelu import reluOBBT, ReLUM
from ml2gurobi.extra.obbt import obbt
from ml2gurobi.sklearn import PipelinePredictor

In [ ]:
# Load the trained network and the examples
pipe = load('MNIST_100_100.joblib')
X = load('MNIST_first100.joblib')

# Choose an example
exampleno = 19
example = X.iloc[exampleno:exampleno+1, :]
ex_prob = pipe.predict_proba(example)
output_shape = ex_prob.shape

In [ ]:
pixels = example.to_numpy().reshape((28, 28))
plt.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
pipe.predict(example)

In [ ]:
sortedidx = np.argsort(ex_prob)[0]

In [ ]:
m = gp.Model()
epsilon = 5

x = m.addMVar(example.shape, lb=0.0, ub=1.0, name='X')
absdiff = m.addMVar(example.shape, lb=0, ub=1, name='dplus')
output = m.addMVar(output_shape, lb=-gp.GRB.INFINITY, name='y')

In [ ]:
m.setObjective(output[0, sortedidx[-2]] - output[0, sortedidx[-1]],
               gp.GRB.MAXIMIZE)

# Bound on the distance to example in norm-1
m.addConstr(absdiff[0, :] >= x[0, :] - example.to_numpy()[0, :])
m.addConstr(absdiff[0, :] >= - x[0, :] + example.to_numpy()[0, :])
m.addConstr(absdiff[0, :].sum() <= epsilon)


In [ ]:
# Change last layer activation to identity
pipe.steps[-1][1].out_activation_ = 'identity'
# Code to add the neural network to the constraints
p2grb = PipelinePredictor(m, pipe, x, output)

# Restore activation
pipe.steps[-1][1].out_activation_ = 'softmax'

In [ ]:
#get number of constraints for formulating neural network
p2grb.get_num('Constrs')

In [ ]:
# Get the list of constraint that were added
p2grb.get_list('Constrs')

In [ ]:
# Gurobi should do something internally about this
obbt(p2grb.steps[-1], activation=reluOBBT('both'))

In [ ]:
m.Params.OutputFlag = 1
m.Params.TimeLimit = 250
m.optimize()

In [ ]:
pixels = x.X.reshape((28, 28))
plt.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
example_mod = pd.DataFrame(data=x.X, columns=example.columns, index=example.index)
pipe.predict(example_mod)

copyright © 2022 Gurobi Optimization, LLC